# Medical Symptom to Disease Prediction NLP

## 1. Install dependencies

In [1]:
# !pip install scikit-learn pandas numpy seaborn matplotlib shap umap-learn transformers torch --quiet
# !pip install transformers datasets evaluate --quiet
# !pip install spacy --quiet
# !python -m spacy download en_core_web_sm --quiet


## 2. Load Symptoms2Disease kaggle dataset

In [4]:
import pandas as pd

# Load the dataset
url = "/content/drive/MyDrive/MedicalProjects/NLP/Disease_Prediction/Dataset/Symptom2Disease.csv"
df = pd.read_csv(url)

# Inspect the dataset
df = df[['label', 'text']]
df.head()

,label,text
0,Psoriasis,I have been experiencing a skin rash on my arm...
1,Psoriasis,"My skin has been peeling, especially on my kne..."
2,Psoriasis,I have been experiencing joint pain in my fing...
3,Psoriasis,"There is a silver like dusting on my skin, esp..."
4,Psoriasis,"My nails have small dents or pits in them, and..."


In [5]:
df.shape

(1200, 2)